<a href="https://colab.research.google.com/github/yellowjs0304/3-min-pytorch/blob/master/04-%ED%8C%A8%EC%85%98_%EC%95%84%EC%9D%B4%ED%85%9C%EC%9D%84_%EA%B5%AC%EB%B6%84%ED%95%98%EB%8A%94_DNN/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 뉴럴넷으로 패션 아이템 구분하기
Fashion MNIST 데이터셋과 앞서 배운 인공신경망을 이용하여 패션아이템을 구분해봅니다.

In [21]:
import torch
# DNN 모델의 재료라 할 수 있는 nn 모듈
import torch.nn as nn
# optimizer를 위한 optim 모듈
import torch.optim as optim
# nn 모듈의 함수 버전 functional 모듈
import torch.nn.functional as F
# torchvision의 데이터셋인 MNIST를 사용하려 아래 라이브러리 임포트
from torchvision import transforms, datasets

In [22]:
# 현재 컴퓨터에서 CUDA(GPU Library)를 이용할 수 있는지 알아보기 위한 Flag
USE_CUDA = torch.cuda.is_available()
# USE_CUDA가 True일 경우, GPU를 할당 그 외엔 CPU 메모리 할당
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [23]:
# Epoch과 Batch_size 선언
EPOCHS = 30
BATCH_SIZE = 64

## 데이터셋 불러오기

In [24]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [25]:
trainset = datasets.FashionMNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.FashionMNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)

train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = True,
)
test_loader = torch.utils.data.DataLoader(
    dataset     = testset,
    batch_size  = BATCH_SIZE,
    shuffle     = True,
)

## 뉴럴넷으로 Fashion MNIST 학습하기

입력 `x` 는 `[배치크기, 색, 높이, 넓이]`로 이루어져 있습니다.
`x.size()`를 해보면 `[64, 1, 28, 28]`이라고 표시되는 것을 보실 수 있습니다.
Fashion MNIST에서 이미지의 크기는 28 x 28, 색은 흑백으로 1 가지 입니다.
그러므로 입력 x의 총 특성값 갯수는 28 x 28 x 1, 즉 784개 입니다.

우리가 사용할 모델은 3개의 레이어를 가진 인공신경망 입니다. 

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        # input data(image) = x 
        # x.shape = torch.Size([64, 1, 28, 28])
        x = x.view(-1, 784)
        # x.shape = torch.Size([64, 784])
        x = F.relu(self.fc1(x))
        # x.shape = torch.Size([64, 256])
        x = F.relu(self.fc2(x))
        # x.shape = torch.Size([64, 128])
        x = self.fc3(x)
        # x.shape = torch.Size([64, 10])
        return x

## 모델 준비하기

`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다.
일반적으로 CPU 1개만 사용할 경우 필요는 없지만,
GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 합니다.
지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠습니다.

In [27]:
# GPU 사용을 위함
model        = Net().to(DEVICE)

#최적화 함수 SGD
optimizer    = optim.SGD(model.parameters(), lr=0.01)

## 학습하기

In [28]:
def train(model, train_loader, optimizer):
    # 모델을 학습 모드로 전환
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target = data.to(DEVICE), target.to(DEVICE)
        # 매 반복(iteration)마다 기울기를 계산하기 위해 zero_grad() 호출
        optimizer.zero_grad()
        #실제 모델의 예측값(output) 받아오기
        output = model(data)
        # 정답 데이터와의 Cross Entropy Loss 계산
        # +)일전엔 2개의 클래스라서 binary cross entropy 였으나, 이번엔 클래스가 10개이니깐 cross entropy 사용
        # loss는 mini batch인 64개의 클래스의 오차 평균값
        loss = F.cross_entropy(output, target)
        # 기울기 계산
        loss.backward()
        # 가중치 수정
        optimizer.step()

## 테스트하기

In [29]:
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()
    # 필요한 변수 초기화
    # Test과정에서의 Loss = test_loss
    # 실제 모델의 예측이 정답과 맞은 횟수 = correct
    test_loss = 0
    correct = 0
    with torch.no_grad(): # 평가 과정에서는 기울기를 계산하지 않으므로, no_grad명시
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 모든 오차 더하기
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            
            # 가장 큰 값을 가진 클래스가 모델의 예측입니다.
            # 예측 클래스(pred)과 정답 클래스를 비교하여 일치할 경우 correct에 1을 더합니다.
            pred = output.max(1, keepdim=True)[1]
            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    #정확도 계산
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 훈련이 되는지 확인해봅시다!

In [30]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

[1] Test Loss: 0.8318, Accuracy: 69.51%
[2] Test Loss: 0.6885, Accuracy: 73.52%
[3] Test Loss: 0.5925, Accuracy: 78.51%
[4] Test Loss: 0.5566, Accuracy: 80.18%
[5] Test Loss: 0.5344, Accuracy: 80.66%
[6] Test Loss: 0.4999, Accuracy: 82.02%
[7] Test Loss: 0.4829, Accuracy: 82.79%
[8] Test Loss: 0.4865, Accuracy: 82.81%
[9] Test Loss: 0.4748, Accuracy: 83.11%
[10] Test Loss: 0.4737, Accuracy: 82.82%
[11] Test Loss: 0.4553, Accuracy: 83.79%
[12] Test Loss: 0.4429, Accuracy: 84.29%
[13] Test Loss: 0.4422, Accuracy: 84.46%
[14] Test Loss: 0.4295, Accuracy: 84.81%
[15] Test Loss: 0.4283, Accuracy: 84.91%
[16] Test Loss: 0.4156, Accuracy: 85.42%
[17] Test Loss: 0.4279, Accuracy: 84.93%
[18] Test Loss: 0.4104, Accuracy: 85.61%
[19] Test Loss: 0.4271, Accuracy: 84.99%
[20] Test Loss: 0.4017, Accuracy: 85.74%
[21] Test Loss: 0.4030, Accuracy: 85.98%
[22] Test Loss: 0.4054, Accuracy: 85.61%
[23] Test Loss: 0.4085, Accuracy: 85.80%
[24] Test Loss: 0.3947, Accuracy: 86.08%
[25] Test Loss: 0.4151, A